In [1]:
import requests
from zipfile import ZipFile
from io import BytesIO

import pandas as pd
from datetime import timedelta
from datetime import datetime

In [29]:
from airflow import DAG
from airflow.operators.python import PythonOperator

Unable to load the config, contains a configuration error.


ValueError: Unable to configure handler 'processor'

In [4]:
TOP_1M_DOMAINS = 'http://s3.amazonaws.com/alexa-static/top-1m.csv.zip'
TOP_1M_DOMAINS_FILE = 'top-1m.csv'

def get_data():
    top_doms = requests.get(TOP_1M_DOMAINS, stream=True)
    zipfile = ZipFile(BytesIO(top_doms.content))
    top_data = zipfile.read(TOP_1M_DOMAINS_FILE).decode('utf-8')

    with open(TOP_1M_DOMAINS_FILE, 'w') as f:
        f.write(top_data)

#Найти топ-10 доменных зон по численности доменов
# домен это все, что до точки. Доменная зона  - .com или .ru и т.д.
def zone_data():
    top_data_df = pd.read_csv(TOP_1M_DOMAINS_FILE, names=['rank', 'domain'])
    top_10_zone_df = top_data_df.domain.str.split('.').str[-1].value_counts().head(10)
    top_10_zone = pd.DataFrame(top_10_zone_df).reset_index()['index']
    with open('top_10_zone.csv', 'w') as f:
            f.write(top_10_zone.to_csv(index=False, header=False))

#Найти домен с самым длинным именем (если их несколько, то взять только 
#первый в алфавитном порядке)
def lengh_domain():
    top_data_df = pd.read_csv(TOP_1M_DOMAINS_FILE, names=['rank', 'domain'])
    lengh_domain = top_data_df.domain.str.split('.').str[0].to_frame()
    lengh_domain['len_name'] = lengh_domain.domain.str.len()
    longest_domain = lengh_domain.sort_values('len_name', ascending=False) \
                                 .head(5) \
                                 .sort_values('domain').iloc[0, 0]
    with open(longest_domain, 'w') as f:
            f.write()

#На каком месте находится домен airflow.com
def site_airflow(domain='airflow.com'):
    top_data_df = pd.read_csv(TOP_1M_DOMAINS_FILE, names=['rank', 'domain'])
    site_airflow = top_data_df.query("domain in @top_data_df.domain")['rank']
    with open(site_airflow, 'w') as f:
            f.write()

def print_data(ds): # передаем глобальную переменную airflow
    with open('top_10_zone.csv', 'r') as f:
        data_zone = f.read()
    print(f'Top domains zones for date {date}')
    print(data_zone)
    
    with open(lengh_domain, 'r') as f:
        lengh_domain = f.read()
    print(f'The longest domain name for date {date}')
    print(lengh_domain)
    
    with open(site_airflow, 'r') as f:
        if site_airflow.empty == True:
            f.read(f'There is no such domain {domain}!')
        else:
            f.read(f'Domain {domain} on the site {site_airflow}')
        date = ds

default_args = {
    'owner': 'z.kapachinskih_16',
    'depends_on_past': False,
    'retries': 2,
    'retry_delay': timedelta(minutes=15),
    'start_date': datetime(2022, 3, 17),
    'schedule_interval': '@daily'
}

dag = DAG('z.kapachinskih_16', default_args=default_args)

t1 = PythonOperator(task_id='get_data',
                    python_callable=get_data,
                    dag=dag)

t2_1 = PythonOperator(task_id='zone_data',
                    python_callable=get_stat,
                    dag=dag)

t2_2 = PythonOperator(task_id='lengh_domain',
                        python_callable=get_stat_com,
                        dag=dag)

t2_3 = PythonOperator(task_id='site_airflow',
                        python_callable=get_stat_com,
                        dag=dag)

t3 = PythonOperator(task_id='print_data',
                    python_callable=print_data,
                    dag=dag)        

t1 >> [t2_1, t2_2] >> t3









NameError: name 'DAG' is not defined

In [5]:
TOP_1M_DOMAINS = 'http://s3.amazonaws.com/alexa-static/top-1m.csv.zip'
TOP_1M_DOMAINS_FILE = 'top-1m.csv'

In [6]:
def site_airflow(domain='airflow.com'):
    top_data_df = pd.read_csv(TOP_1M_DOMAINS_FILE, names=['rank', 'domain'])
    site_airflow = top_data_df.query("domain in @top_data_df.domain")['rank']
    with open(site_airflow, 'w') as f:
            f.write()

In [7]:
with open(site_airflow, 'r') as f:
        if site_airflow.empty == True:
            f.read(f'There is no such domain {domain}!')
        else:
            f.read(f'Domain {domain} on the site {site_airflow}')

TypeError: expected str, bytes or os.PathLike object, not function

In [9]:
top_data_df = pd.read_csv(TOP_1M_DOMAINS_FILE, names=['rank', 'domain'])
site_airflow = top_data_df.query("domain in @top_data_df.domain")['rank']
site_airflow

FileNotFoundError: [Errno 2] No such file or directory: 'top-1m.csv'